# Get cleaner annotations and save them as json

In [4]:
import os
from glob import glob
import csv
import json
from tqdm.notebook import tqdm

annotation_path = {}
annotation_path['train'] = "/home/tk/repos/MELD/data/MELD/train_sent_emo.csv" 
annotation_path['dev'] = "/home/tk/repos/MELD/data/MELD/dev_sent_emo.csv" 
annotation_path['test'] = "/home/tk/repos/MELD/data/MELD/test_sent_emo.csv" 

        
vids_dir = {}
vids_dir['train'] = "/home/tk/datasets/MELD/MELD.Raw/train/train_splits/" 
vids_dir['dev'] = "/home/tk/datasets/MELD/MELD.Raw/dev/dev_splits_complete/" 
vids_dir['test'] = "/home/tk/datasets/MELD/MELD.Raw/test/output_repeated_splits_test/"


annotations = {}
for DATATYPE in ['train', 'dev', 'test']:
    with open(annotation_path[DATATYPE]) as f:
        reader = csv.reader(f)
        annotations[DATATYPE] = list(reader)

vid2anno = {}
weird = 0
for DATATYPE in tqdm(['train', 'dev', 'test']):
    vid2anno[DATATYPE] = {}
    for row in tqdm(annotations[DATATYPE][1:]):
        SrNo, Utterance, Speaker, Emotion, Sentiment, Dialogue_ID,\
            Utterance_ID, Season, Episode, StartTime, EndTime = row

        if f"dia{Dialogue_ID}_utt{Utterance_ID}.mp4" not in str(os.listdir(vids_dir[DATATYPE])):
            weird+=1
            continue
        vid2anno[DATATYPE][f"dia{Dialogue_ID}_utt{Utterance_ID}"] = \
            {'SrNo': SrNo, 
            'Utterance': Utterance, 
            'Speaker': Speaker, 
            'Emotion': Emotion, 
            'Sentiment': Sentiment, 
            'Dialogue_ID': Dialogue_ID,
            'Utterance_ID': Utterance_ID, 
            'Season': Season, 
            'Episode': Episode, 
            'StartTime': StartTime, 
            'EndTime':EndTime}
                
print(f"the number of video that didn't match is : {weird}")

SAVE_AT = "/home/tk/datasets/MELD/visual-features/entire-dataset"
with open(os.path.join(SAVE_AT, 'datasets.json'), 'w', encoding='utf8') as stream:
    json.dump(vid2anno, stream, ensure_ascii=False)




the number of video that didn't match is : 1


# Instantiate the three classes

In [5]:
from cltl_face_all.face_alignment import FaceDetection
from cltl_face_all.arcface import ArcFace
from cltl_face_all.agegender import AgeGender

fd = FaceDetection(device='cuda', face_detector='sfd')
ag = AgeGender(device='cpu')
af = ArcFace(device='cpu')

[*] load ckpt from /home/tk/.virtualenvs/dev-python-3.7/lib/python3.7/site-packages/cltl_face_all/arcface/./pretrained_models/arc_res50/e_8_b_40000.ckpt


# Run over the images

In [12]:
import av
import numpy as np
import os

for DATATYPE in tqdm(['train', 'dev', 'test']):
    os.makedirs(os.path.join(SAVE_AT, DATATYPE), exist_ok=True)

    for diautt, anno in tqdm(vid2anno[DATATYPE].items()):
        to_dump = {}
        vidpath = os.path.join(vids_dir[DATATYPE], diautt) + '.mp4'

        container = av.open(vidpath)

        save_full_path = os.path.join(SAVE_AT, DATATYPE, diautt) + '.npy'

        if os.path.isfile(save_full_path) and os.path.getsize(save_full_path) > 1024:
            continue

        for frame in tqdm(container.decode(video=0)):
            idx = frame.index
            to_dump[idx] = []
            numpy_RGB = np.array(frame.to_image())
            batch = numpy_RGB[np.newaxis, ...]
            bboxes = fd.detect_faces(batch)
            # print(f"number of faces detected in the frame {idx} is {len(bboxes[0])}")

            if len(bboxes[0]) == 0:
                continue

            landmarks = fd.detect_landmarks(batch, bboxes)

            faces = fd.crop_and_align(batch, bboxes, landmarks)
            faces = np.concatenate(faces, axis=0)
            embeddings = af.predict(faces)
            ages, genders = ag.predict(faces)

            # print(len(bboxes[0]), len(landmarks[0]), len(ages), len(genders), len(embeddings), len(faces))
            for bb, lm, a, g, emb, f in zip(bboxes[0], landmarks[0], ages, genders, embeddings, faces):
                x1, y1, x2, y2, prob = bb

                to_append = {'bbox': bb, 
                            'landmark': lm, 
                            'age': a,
                            'gender': g,
                            'embedding': emb,
                            'face': f}

                to_dump[idx].append(to_append)
            # print(len(to_dump), idx+1)

        np.save(save_full_path, to_dump)

KeyboardInterrupt: 